In [ ]:
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import xarray as xr

from sdm_eurec4a import RepositoryPath, conversions
from sdm_eurec4a.input_processing import transfer

from sdm_eurec4a.reductions import shape_dim_as_dataarray
from sdm_eurec4a.visulization import (
    adjust_lightness_array,
    set_custom_rcParams,
    handler_map_alpha,
)

In [ ]:
def set_xticks_time(ax):
    xticks = [0, 500, 1000]
    ax.set_xticks(xticks)


def set_yticks_height(ax):
    yticks = [0, 500, 1000, 1500, 2000]
    ax.set_yticks(yticks)


def set_yticks_height_km(ax):
    yticks = [0, 0.5, 1, 1.5, 2]
    ax.set_yticks(yticks)


def set_logxticks_meter(ax):
    xticks = [1e-6, 1e-3]
    xticklabels = [r"$10^{-6}$", r"$10^{-3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logxticks_micrometer(ax):
    xticks = [1e-3, 1e0, 1e3]
    xticklabels = [r"$10^{-3}$", r"$10^{0}$", r"$10^{3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logtyticks_psd(ax):
    yticks = [1e0, 1e6]
    yticklabels = [r"$10^0$", r"$10^6$"]
    ax.set_yticks(yticks, yticklabels)


def set_yticks_lwc(ax):
    ax.set_yticks([0, 0.1, 0.2])

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
plt.rcParams.update(
    {
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.spines.left": False,
        "axes.spines.bottom": False,
    }
)

dark_colors = adjust_lightness_array(default_colors, amount=0.5)

repo_path = RepositoryPath("levante")()
print(repo_path)

# data_subdir = "output_v3.2/stationary_no_physics"
# data_subdir = "output_v3.2/stationary_condensation"
data_subdir = "output_v3.2"

subdata_dirs = dict(
    no_physics=dict(
        path=Path("/home/m/m301096/CLEO/data/")
        / data_subdir
        / "stationary_no_physics"
        / "combined/eulerian_dataset_combined.nc",
        microphysics="No physics",
    ),
    condensation=dict(
        path=Path("/home/m/m301096/CLEO/data/")
        / data_subdir
        / "stationary_condensation"
        / "combined/eulerian_dataset_combined.nc",
        microphysics="Condensation/Evaporation",
    ),
    collision_condensation=dict(
        path=Path("/home/m/m301096/CLEO/data/")
        / data_subdir
        / "stationary_collision_condensation"
        / "combined/eulerian_dataset_combined.nc",
        microphysics="Collision and cond./evap.",
    ),
)

# THE PATH TO THE SCRIPT DIRECTORY
script_dir = Path("/home/m/m301096/repositories/sdm-eurec4a/notebooks/presentation/results/")
print(script_dir)


fig_dir = repo_path / "results" / script_dir.relative_to(repo_path) / data_subdir
print(fig_dir)

fig_dir.mkdir(parents=True, exist_ok=True)

/home/m/m301096/repositories/sdm-eurec4a
/home/m/m301096/repositories/sdm-eurec4a/notebooks/presentation/results
/home/m/m301096/repositories/sdm-eurec4a/results/notebooks/presentation/results/output_v3.2


## Load datasets

In [ ]:
for key in subdata_dirs:
    ds = xr.open_dataset(subdata_dirs[key]["path"])
    # ds = ds.sel(cloud_id = [18, 301])
    ds.attrs.update(micro_physics=subdata_dirs[key]["microphysics"])
    subdata_dirs[key]["combined_output"] = ds

## Calculations

### Calculate masks and coord3 and volume

In [ ]:
def add_variables(combined_output: xr.Dataset):
    combined_output["sub_cloud_layer_mask"] = combined_output["gridbox"] < combined_output["max_gridbox"]
    combined_output["sub_cloud_layer_mask"].attrs.update(
        long_name="Sub Cloud Layer Mask",
        description="Boolean mask indicating if the gridbox is part of the sub cloud layer",
        units="1",
    )

    combined_output["sub_cloud_layer_no_bottom"] = (combined_output["sub_cloud_layer_mask"]) & (
        combined_output["gridbox"] > 0
    )
    combined_output["sub_cloud_layer_no_bottom"].attrs.update(
        long_name="Sub Cloud Layer Mask without bottom",
        description="Boolean mask indicating if the gridbox is part of the sub cloud layer and not the bottom gridbox",
        units="1",
    )

    combined_output["pseudo_coord3"] = combined_output["gridbox"] * 20
    combined_output["pseudo_coord3"].attrs.update(
        long_name="pseudo Coordinate 3",
        description="pseudo Coordinate 3. Which is the gridbox number times 20m. It thus is wrong for the top gridbox for each cloud. This can be identified by the variable 'max_gridbox'. So use the mask 'sub_cloud_layer_mask' to identify the sub cloud layer.",
        units="m",
    )

    combined_output["pseudo_coord3_normalized"] = (
        combined_output["pseudo_coord3"]
        / combined_output["pseudo_coord3"][combined_output["max_gridbox"] - 1]
    )

    Z = np.nan_to_num(
        x=(combined_output["pseudo_coord3"].where(combined_output["sub_cloud_layer_mask"])),
        nan=0,
    )
    Z_max = np.nan_to_num(
        x=(
            combined_output["pseudo_coord3"].where(
                combined_output["gridbox"] == combined_output["max_gridbox"]
            )
        )
        * 0
        + combined_output["max_gridbox"] * 20
        + (100 - 20) / 2,
        nan=0,
    )
    Z = Z + Z_max
    combined_output["pseudo_coord3_full"] = (("gridbox", "cloud_id"), Z)
    combined_output["pseudo_coord3_full"] = combined_output["pseudo_coord3_full"].where(
        combined_output["gridbox"] <= combined_output["max_gridbox"]
    )
    combined_output["pseudo_coord3_full"].attrs.update(
        long_name="pseudo Coordinate 3 Full",
        description="pseudo Coordinate 3. Each gridbox below cloud layer is 20m thick and cloud layer is 100m thick.",
        units="m",
    )

    combined_output["pseudo_gribox_volume"] = combined_output["pseudo_coord3"] * 0 + 20**3
    combined_output["pseudo_gribox_volume"].attrs.update(
        long_name="Simple Pseudo Gridbox Volume",
        description="pseudo Gridbox Volume. Which is the gridbox volume for each gridbox in the sub cloud layer. It is 20m x 20m x 20m for each gridbox.",
        units="m^3",
    )

    combined_output["pseudo_gribox_volume_full"] = (
        ("gridbox", "cloud_id"),
        np.nan_to_num(
            x=(
                combined_output["pseudo_gribox_volume"].where(
                    combined_output["gridbox"] != combined_output["max_gridbox"]
                )
            ),
            nan=20 * 20 * 100,
        ),
    )
    combined_output["pseudo_gribox_volume_full"] = combined_output["pseudo_gribox_volume_full"].where(
        combined_output["gridbox"] <= combined_output["max_gridbox"]
    )
    combined_output["pseudo_gribox_volume"].attrs.update(
        long_name="pseudo Gridbox Volume",
        description="pseudo Gridbox Volume. Which is the gridbox volume for each gridbox in the sub cloud layer. It is 20m x 20m x 20m for each gridbox in subcloud layer. 20m x 20m x 100m in cloud layer. nan for gridboxes above cloud layer.",
        units="m^3",
    )
    vertical_profile_mass = (
        combined_output["mass_represented"].sel(time=slice(400, None)).sum("radius_bins").mean("time")
    )
    vertical_profile_mass = vertical_profile_mass.where(combined_output["sub_cloud_layer_mask"])
    vertical_profile_mass = 1e3 * vertical_profile_mass / combined_output["pseudo_gribox_volume"]
    vertical_profile_mass.attrs.update(
        long_name="Mass",
        description="Vertical Profile of Mass. The mass is averaged over time and radius bins and then divided by the gridbox volume.",
        units=r"$g m^{-3}$",
    )
    combined_output["vertical_profile_mass"] = vertical_profile_mass

    vertical_profile_mass_std = (
        combined_output["mass_represented"].sel(time=slice(400, None)).sum("radius_bins").std("time")
    )
    vertical_profile_mass_std = vertical_profile_mass_std.where(combined_output["sub_cloud_layer_mask"])
    vertical_profile_mass_std = 1e3 * vertical_profile_mass_std / combined_output["pseudo_gribox_volume"]
    vertical_profile_mass_std.attrs.update(
        long_name="Mass",
        description="Vertical Profile of Mass. The mass is averaged over time and radius bins and then divided by the gridbox volume.",
        units=r"$g m^{-3}$",
    )
    combined_output["vertical_profile_mass_std"] = vertical_profile_mass_std

    combined_output["top_subcloud_mass"] = (
        combined_output["vertical_profile_mass"]
        .where(combined_output["gridbox"] == combined_output["max_gridbox"] - 1)
        .mean("gridbox")
    )
    combined_output["top_subcloud_mass"].attrs.update(
        long_name="Top Subcloud Mass",
        description="Mass of the top subcloud layer. The mass is averaged over time and radius bins and then divided by the gridbox volume.",
        units=r"$g m^{-3}$",
    )

    combined_output["bottom_subcloud_mass"] = combined_output["vertical_profile_mass"].sel(gridbox=0)
    combined_output["bottom_subcloud_mass"].attrs.update(
        long_name="Bottom Subcloud Mass",
        description="Mass of the bottom subcloud layer. The mass is averaged over time and radius bins and then divided by the gridbox volume.",
        units=r"$g m^{-3}$",
    )

    vertical_profile_mass_fraction = (
        100
        * combined_output["vertical_profile_mass"]
        / combined_output["vertical_profile_mass"].sel(gridbox=combined_output["max_gridbox"] - 1)
    )
    vertical_profile_mass_fraction.attrs.update(
        long_name="Mass Fraction",
        description="Vertical Profile of Mass Fraction. The mass is divided by the mass of the topmost sub-cloud-layer gridbox.",
        units="%",
    )
    combined_output["vertical_profile_mass_fraction"] = vertical_profile_mass_fraction

    vertical_profile_mass_fraction_std = (
        100
        * combined_output["vertical_profile_mass_std"]
        / combined_output["vertical_profile_mass"].sel(gridbox=combined_output["max_gridbox"] - 1)
    )
    vertical_profile_mass_fraction_std.attrs.update(
        long_name="Mass Fraction",
        description="Vertical Profile of Mass Fraction. The mass is divided by the mass of the topmost sub-cloud-layer gridbox.",
        units="%",
    )
    combined_output["vertical_profile_mass_fraction_std"] = vertical_profile_mass_fraction_std

    vertical_profile_mass_diff = (
        combined_output["mass_difference"].sel(time=slice(400, None)).sum("radius_bins").mean("time")
    )
    vertical_profile_mass_diff = vertical_profile_mass_diff.where(
        combined_output["sub_cloud_layer_no_bottom"]
    )
    vertical_profile_mass_diff = (
        1e3 * vertical_profile_mass_diff / combined_output["pseudo_gribox_volume"]
    )
    vertical_profile_mass_diff.attrs.update(
        long_name="Mass Difference",
        description="Vertical Profile of Mass Difference. The mass difference is averaged over time and radius bins and then divided by the gridbox volume. ",
        units=r"$g m^{-3} s^{-1}$",
    )
    combined_output["vertical_profile_mass_diff"] = vertical_profile_mass_diff

    vertical_evaporation_fraction = 100 * vertical_profile_mass_diff / vertical_profile_mass
    vertical_evaporation_fraction.attrs.update(
        long_name="Vertical Evaporation Fraction",
        description="Vertical Evaporation Fraction. The mass difference is divided by the mass to get the fraction of mass which evaporates in the sub cloud layer.",
        units=r"$\%$",
    )
    combined_output["vertical_evaporation_fraction"] = vertical_evaporation_fraction

    # total values
    combined_output["mass_difference_total"] = (
        combined_output["vertical_profile_mass_diff"] * combined_output["pseudo_gribox_volume"]
    ).sum("gridbox", keep_attrs=True) / (
        combined_output["pseudo_gribox_volume"]
        .where(combined_output["sub_cloud_layer_no_bottom"])
        .sum("gridbox")
    )
    combined_output["mass_difference_total"].attrs.update(
        long_name="Total Mass Difference",
        description="Total Mass Difference. The mass difference is summed over all gridboxes and divided by the total volume of the sub cloud layer.",
        units=r"$g m^{-3} s^{-1}$",
    )

    combined_output["mass_total"] = combined_output["vertical_profile_mass"].sum("gridbox")
    combined_output["mass_total"].attrs.update(
        long_name="Total Mass",
        description="Total Mass. The mass is summed over all gridboxes of the sub cloud layer.",
        units=r"$g$",
    )

    combined_output["mass_fraction_total"] = vertical_profile_mass_fraction.sel(gridbox=1)
    combined_output["mass_fraction_total"].attrs.update(
        long_name="Total Mass Fraction",
        description="Total Mass Fraction. The mass fraction of the mass that reaches the bottom.",
        units="%",
    )

    combined_output["liquid_water_content"] = 1e3 * (
        combined_output["mass_represented"] / combined_output["pseudo_gribox_volume_full"]
    )
    combined_output["liquid_water_content"].attrs.update(
        long_name="Liquid Water Content",
        description="Liquid Water Content per gridbox for the sub cloud layer.",
        units=r"$g m^{-3}$",
    )

    combined_output["liquid_water_content_smooth"] = (
        combined_output["liquid_water_content"].rolling(time=50, center=True).mean()
    )
    combined_output["liquid_water_content_smooth"].attrs.update(
        long_name="Liquid Water Content Smooth",
        description="Smoothed Liquid Water Content per gridbox for the sub cloud layer.",
        units=r"$g m^{-3}$",
    )

    vertical_profile_lwc = (
        combined_output["liquid_water_content"]
        .sel(time=slice(400, None))
        .sum("radius_bins", keep_attrs=True)
        .mean("time", keep_attrs=True)
    )
    vertical_profile_lwc = vertical_profile_lwc.where(
        combined_output["gridbox"] <= combined_output["max_gridbox"]
    )
    vertical_profile_lwc.attrs.update(
        long_name="Liquid Water Content",
        description="Vertical Profile of liquid water content.",
    )
    combined_output["vertical_profile_lwc"] = vertical_profile_lwc.where(
        combined_output["sub_cloud_layer_mask"]
    )
    combined_output["vertical_profile_lwc_including_cloud"] = vertical_profile_lwc

    vertical_profile_lwc_fraction = combined_output["vertical_profile_lwc"] - combined_output[
        "vertical_profile_lwc"
    ].sel(gridbox=combined_output["max_gridbox"] - 1)
    vertical_profile_lwc_fraction.attrs.update(
        long_name="Liquid Water Content fraction",
        description="Vertical Profile of liquid water content fraction.",
        units=combined_output["liquid_water_content"].units,
    )
    combined_output["vertical_profile_lwc_fraction"] = vertical_profile_lwc_fraction

In [ ]:
for key in subdata_dirs:
    add_variables(subdata_dirs[key]["combined_output"])

### Calculate vertical profiles and total numbers

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))
ax_microlabel = ax.twinx()
ax_microlabel.set_yticks([])

idx = 0
for key, linestyle in zip(["condensation", "collision_condensation", "no_physics"], ["-", "--", ":"]):
    combined_output = subdata_dirs[key]["combined_output"]
    combined_output = combined_output.sel(cloud_id=combined_output["max_gridbox"] < 40)

    for cloud_id, color in zip([301, 18], ["r", "b"]):
        if idx == 0:
            label = f"cloud {cloud_id}"
        else:
            label = None
        ax.plot(
            combined_output.sel(cloud_id=cloud_id)["vertical_profile_mass_fraction"],
            combined_output["pseudo_coord3"],
            c=color,
            alpha=1,
            label=label,
            linestyle=linestyle,
            zorder=3,
        )

    ax_microlabel.plot(
        np.nan,
        np.nan,
        c="k",
        alpha=0,
        label=f"{subdata_dirs[key]['microphysics']}",
        linestyle=linestyle,
    )
    idx += 1


xlabel = (
    combined_output["vertical_profile_mass_fraction"].long_name
    + f" {combined_output['vertical_profile_mass_fraction'].attrs['units']}"
)
ax.set_xlabel(xlabel)
# # ax.set_title(ind_cloud_output1[vertical_profile_mass_fraction].long_name)

set_yticks_height(ax)
ax.set_ylim(0, 800)

ax.set_ylabel("Height $m$")
ax.set_xticks([60, 80, 100])
ax.set_xlim(60, 105)
ax.axvline(100, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")
ax_microlabel.legend(handler_map=handler_map_alpha(), loc="upper center")
# # ax.set_title("Mass fraction relative to top sub-cloud layer")

In [ ]:
fig.savefig(fig_dir / "mf_relative_scl_selected.svg")

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))
ax_microlabel = ax.twinx()
ax_microlabel.set_yticks([])

idx = 0
for key, linestyle in zip(["condensation", "collision_condensation", "no_physics"], ["-", "--", ":"]):
    combined_output = subdata_dirs[key]["combined_output"]
    combined_output = combined_output.sel(cloud_id=combined_output["max_gridbox"] < 40)

    for cloud_id, color in zip([301, 18], ["r", "b"]):
        if idx == 0:
            label = f"cloud {cloud_id}"
        else:
            label = None
        ax.plot(
            combined_output.sel(cloud_id=cloud_id)["vertical_profile_mass_fraction"],
            combined_output["pseudo_coord3"],
            c=color,
            alpha=1,
            label=label,
            linestyle=linestyle,
            zorder=3,
        )

    ax_microlabel.plot(
        np.nan,
        np.nan,
        c="k",
        alpha=0,
        label=f"{subdata_dirs[key]['microphysics']}",
        linestyle=linestyle,
    )
    idx += 1

    ax.plot(
        combined_output["vertical_profile_mass_fraction"].quantile(0.5, "cloud_id"),
        combined_output["pseudo_coord3"],
        c="k",
        alpha=1,
        linestyle=linestyle,
        zorder=1,
    )
    ax.fill_betweenx(
        combined_output["pseudo_coord3"],
        combined_output["vertical_profile_mass_fraction"].quantile(0.33, "cloud_id"),
        combined_output["vertical_profile_mass_fraction"].quantile(0.66, "cloud_id"),
        color="k",
        alpha=0.1,
        linestyle=linestyle,
        zorder=1,
    )


xlabel = (
    combined_output["vertical_profile_mass_fraction"].long_name
    + f" {combined_output['vertical_profile_mass_fraction'].attrs['units']}"
)
ax.set_xlabel(xlabel)
# # ax.set_title(ind_cloud_output1[vertical_profile_mass_fraction].long_name)

set_yticks_height(ax)
ax.set_ylim(0, 800)

ax.set_ylabel("Height $m$")
ax.set_xticks([60, 80, 100])
ax.set_xlim(60, 105)
ax.axvline(100, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")
ax_microlabel.legend(handler_map=handler_map_alpha(), loc="upper center")
# # ax.set_title("Mass fraction relative to top sub-cloud layer")

/work/um1487/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


In [ ]:
fig.savefig(fig_dir / "mf_relative_scl_mean.svg")

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))
ax_microlabel = ax.twinx()
ax_microlabel.set_yticks([])

idx = 0
for key, linestyle in zip(["condensation", "collision_condensation", "no_physics"], ["-", "--", ":"]):
    combined_output = subdata_dirs[key]["combined_output"]
    combined_output = combined_output.sel(cloud_id=combined_output["max_gridbox"] < 40)

    for cloud_id, color in zip([301, 18], ["r", "b"]):
        if idx == 0:
            label = f"cloud {cloud_id}"
        else:
            label = None
        ax.plot(
            combined_output.sel(cloud_id=cloud_id)["vertical_profile_mass_fraction"],
            combined_output["pseudo_coord3"],
            c=color,
            alpha=1,
            label=label,
            linestyle=linestyle,
            zorder=3,
        )

    ax_microlabel.plot(
        np.nan,
        np.nan,
        c="k",
        alpha=0,
        label=f"{subdata_dirs[key]['microphysics']}",
        linestyle=linestyle,
    )
    idx += 1
for key, linestyle in zip(["condensation"], ["-"]):
    combined_output = subdata_dirs[key]["combined_output"]
    combined_output = combined_output.sel(cloud_id=combined_output["max_gridbox"] < 40)

    ax.plot(
        combined_output["vertical_profile_mass_fraction"].T,
        combined_output["pseudo_coord3"],
        c="grey",
        alpha=0.2,
        linestyle=linestyle,
        zorder=1,
    )

xlabel = (
    combined_output["vertical_profile_mass_fraction"].long_name
    + f" {combined_output['vertical_profile_mass_fraction'].attrs['units']}"
)
ax.set_xlabel(xlabel)
# # ax.set_title(ind_cloud_output1[vertical_profile_mass_fraction].long_name)

set_yticks_height(ax)
ax.set_ylim(0, 800)

ax.set_ylabel("Height $m$")
ax.set_xticks([60, 80, 100])
ax.set_xlim(60, 105)
ax.axvline(100, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")
ax_microlabel.legend(handler_map=handler_map_alpha(), loc="upper center")
# # ax.set_title("Mass fraction relative to top sub-cloud layer")

In [ ]:
fig.savefig(fig_dir / "mf_relative_scl_all.svg")

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))
ax_microlabel = ax.twinx()
ax_microlabel.set_yticks([])

idx = 0
for key, linestyle in zip(["condensation", "collision_condensation", "no_physics"], ["-", "--", ":"]):
    combined_output = subdata_dirs[key]["combined_output"]
    combined_output = combined_output.sel(cloud_id=combined_output["max_gridbox"] < 40)

    for cloud_id, color in zip([301, 18], ["r", "b"]):
        if idx == 0:
            label = f"cloud {cloud_id}"
        else:
            label = None
        ax.plot(
            combined_output.sel(cloud_id=cloud_id)["vertical_profile_lwc_fraction"],
            combined_output["pseudo_coord3"],
            c=color,
            alpha=1,
            label=label,
            linestyle=linestyle,
            zorder=3,
        )

    ax_microlabel.plot(
        np.nan,
        np.nan,
        c="k",
        alpha=0,
        label=f"{subdata_dirs[key]['microphysics']}",
        linestyle=linestyle,
    )
    idx += 1

    ax.plot(
        combined_output["vertical_profile_lwc_fraction"].quantile(0.5, "cloud_id"),
        combined_output["pseudo_coord3"],
        c="k",
        alpha=1,
        linestyle=linestyle,
        zorder=1,
    )
    ax.fill_betweenx(
        combined_output["pseudo_coord3"],
        combined_output["vertical_profile_lwc_fraction"].quantile(0.33, "cloud_id"),
        combined_output["vertical_profile_lwc_fraction"].quantile(0.66, "cloud_id"),
        color="k",
        alpha=0.1,
        linestyle=linestyle,
        zorder=1,
    )


xlabel = (
    combined_output["vertical_profile_lwc_fraction"].long_name
    + f" {combined_output['vertical_profile_lwc_fraction'].attrs['units']}"
)
ax.set_xlabel(xlabel)
# # ax.set_title(ind_cloud_output1[vertical_profile_mass_fraction].long_name)

set_yticks_height(ax)
ax.set_ylim(0, 800)

ax.set_ylabel("Height $m$")
ax.set_xticks(
    [
        0,
        -0.01,
    ]
)
ax.set_xlim(-0.01, 0.002)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)

ax.legend(loc="upper left")
ax_microlabel.legend(handler_map=handler_map_alpha(), loc="upper center")
# # ax.set_title("Mass fraction relative to top sub-cloud layer")

In [ ]:
fig.savefig(fig_dir / "lwc_relative_scl_mean.svg")

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))
ax_microlabel = ax.twinx()
ax_microlabel.set_yticks([])

idx = 0
for key, linestyle in zip(["condensation", "collision_condensation", "no_physics"], ["-", "--", ":"]):
    combined_output = subdata_dirs[key]["combined_output"]
    combined_output = combined_output.sel(cloud_id=combined_output["max_gridbox"] < 40)

    for cloud_id, color in zip([301, 18], ["r", "b"]):
        if idx == 0:
            label = f"cloud {cloud_id}"
        else:
            label = None
        ax.plot(
            combined_output.sel(cloud_id=cloud_id)["vertical_profile_lwc_fraction"],
            combined_output["pseudo_coord3"],
            c=color,
            alpha=1,
            label=label,
            linestyle=linestyle,
            zorder=3,
        )

    ax_microlabel.plot(
        np.nan,
        np.nan,
        c="k",
        alpha=0,
        label=f"{subdata_dirs[key]['microphysics']}",
        linestyle=linestyle,
    )
    idx += 1
for key, linestyle in zip(["condensation"], ["-"]):
    combined_output = subdata_dirs[key]["combined_output"]
    combined_output = combined_output.sel(cloud_id=combined_output["max_gridbox"] < 40)

    ax.plot(
        combined_output["vertical_profile_lwc_fraction"].T,
        combined_output["pseudo_coord3"],
        c="grey",
        alpha=0.2,
        linestyle=linestyle,
        zorder=1,
    )

xlabel = (
    combined_output["vertical_profile_lwc_fraction"].long_name
    + f" {combined_output['vertical_profile_lwc_fraction'].attrs['units']}"
)
ax.set_xlabel(xlabel)
# # ax.set_title(ind_cloud_output1[vertical_profile_mass_fraction].long_name)

set_yticks_height(ax)
ax.set_ylim(0, 800)

ax.set_ylabel("Height $m$")
ax.set_xticks(
    [
        0,
        -0.01,
    ]
)
ax.set_xlim(-0.01, 0.002)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")
ax_microlabel.legend(handler_map=handler_map_alpha(), loc="upper center")
# # ax.set_title("Mass fraction relative to top sub-cloud layer")

In [ ]:
fig.savefig(fig_dir / "lwc_relative_scl_all.svg")

In [ ]:
for key, linestyle in zip(["condensation", "collision_condensation", "no_physics"], ["-", "--", ":"]):
    combined_output = subdata_dirs[key]["combined_output"]
    combined_output = combined_output.sel(cloud_id=combined_output["max_gridbox"] < 40)
    combined_output = combined_output.sel(gridbox=slice(None, 40))
    combined_output = combined_output.sel(time=slice(400, None))
    data = (
        combined_output["mass_difference"].sum("radius_bins").mean("time")
        / combined_output["pseudo_gribox_volume_full"]
    )
    data = data.where(combined_output["sub_cloud_layer_no_bottom"])
    plt.plot(
        data.median("cloud_id"),
        data.gridbox,
        label=key,
        linestyle=linestyle,
        color="k",
    )
    plt.fill_betweenx(
        data.gridbox,
        data.quantile(0.33, "cloud_id"),
        data.quantile(0.66, "cloud_id"),
        label=key,
        linestyle=linestyle,
        alpha=0.1,
        color="k",
    )

/work/um1487/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/work/um1487/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/work/um1487/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=1, figsize=(15, 4.5))

idx = 0
for key, linestyle, color in zip(
    ["condensation", "collision_condensation", "no_physics"], ["-", "-", "-"], ["k", "k", "k"]
):
    ax = axs[idx]
    combined_output = subdata_dirs[key]["combined_output"]
    combined_output = combined_output.sel(cloud_id=combined_output["max_gridbox"] < 40)
    combined_output = combined_output.sel(gridbox=slice(None, 40))
    combined_output = combined_output.sel(time=slice(400, None))
    data = (
        1e3
        * combined_output["mass_difference_timestep"].sum("radius_bins").mean("time")
        / combined_output["pseudo_gribox_volume_full"]
    )
    data = data.where(combined_output["sub_cloud_layer_no_bottom"])
    data = data.sortby(-data["gridbox"]).cumsum(dim="gridbox")
    ax.plot(
        data.T,
        data.gridbox,
        label=key,
        linestyle=linestyle,
        color=color,
        alpha=0.1,
    )
    ax.plot(
        data.quantile(0.5, "cloud_id"),
        data.gridbox,
        label=key,
        linestyle=linestyle,
        alpha=1,
        color=color,
    )
    ax.set_title(key)

    idx += 1

for ax in axs:
    # ax.set_xlim(, 0.1e-4)
    ax.set_xlabel("Number of Superdroplets")
    ax.set_ylabel("gridbox")

# fig.savefig("number_superdroplets.svg")

In [ ]:
for key in subdata_dirs:
    combined_output = subdata_dirs[key]["combined_output"]
    combined_output = combined_output.sel(cloud_id=combined_output["max_gridbox"] < 40)
    combined_output = combined_output.sel(gridbox=slice(None, 40))
    combined_output = combined_output.sel(time=slice(400, None))
    data = (
        1e3 * combined_output["mass_difference_timestep"] / combined_output["pseudo_gribox_volume_full"]
    )
    # data = data.where(combined_output['sub_cloud_layer_no_bottom'])
    cumsum_loss = data.sortby(100 - data["gridbox"]).cumsum(dim="gridbox")
    cumsum_loss = cumsum_loss.sortby("gridbox")
    cumsum_loss.attrs.update(
        long_name="Cumulative Mass Loss",
        description="Cumulative Mass Loss. The mass loss is summed over all gridboxes from the bottom to the top.",
        units=r"$g m^{-3}$",
    )
    combined_output["cumsum_loss"] = cumsum_loss

    vertical_cumsum_loss = combined_output["cumsum_loss"].sum("radius_bins").mean("time")
    vertical_cumsum_loss = vertical_cumsum_loss.where(combined_output["sub_cloud_layer_mask"])
    vertical_cumsum_loss.attrs.update(
        long_name="Cumulative Mass Loss",
        description="Cumulative Mass Loss. The mass loss is summed over all gridboxes from the bottom to the top.",
        units=r"$g m^{-3}$",
    )
    combined_output["vertical_cumsum_loss"] = vertical_cumsum_loss
    subdata_dirs[key]["combined_output"] = combined_output

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))
ax_microlabel = ax.twinx()
ax_microlabel.set_yticks([])

idx = 0
for key, linestyle in zip(["condensation", "collision_condensation", "no_physics"], ["-", "--", ":"]):
    combined_output = subdata_dirs[key]["combined_output"]
    combined_output = combined_output.sel(cloud_id=combined_output["max_gridbox"] < 40)

    for cloud_id, color in zip([301, 18], ["r", "b"]):
        if idx == 0:
            label = f"cloud {cloud_id}"
        else:
            label = None
        ax.plot(
            combined_output.sel(cloud_id=cloud_id)["vertical_cumsum_loss"],
            combined_output["pseudo_coord3"],
            c=color,
            alpha=1,
            label=label,
            linestyle=linestyle,
            zorder=3,
        )

    ax_microlabel.plot(
        np.nan,
        np.nan,
        c="k",
        alpha=0,
        label=f"{subdata_dirs[key]['microphysics']}",
        linestyle=linestyle,
    )
    idx += 1

    ax.plot(
        combined_output["vertical_cumsum_loss"].quantile(0.5, "cloud_id"),
        combined_output["pseudo_coord3"],
        c="k",
        alpha=1,
        linestyle=linestyle,
        zorder=1,
    )
    ax.fill_betweenx(
        combined_output["pseudo_coord3"],
        combined_output["vertical_cumsum_loss"].quantile(0.33, "cloud_id"),
        combined_output["vertical_cumsum_loss"].quantile(0.66, "cloud_id"),
        color="k",
        alpha=0.1,
        linestyle=linestyle,
        zorder=1,
    )


xlabel = (
    combined_output["vertical_cumsum_loss"].long_name
    + f" {combined_output['vertical_cumsum_loss'].attrs['units']}"
)
ax.set_xlabel(xlabel)
# # ax.set_title(ind_cloud_output1[vertical_profile_mass_fraction].long_name)

set_yticks_height(ax)
ax.set_ylim(0, 800)

ax.set_ylabel("Height $m$")
ax.set_xticks(
    [
        0,
        -0.001,
        -0.002,
    ]
)
ax.set_xlim(-3e-3, 1e-3)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)

ax.legend(loc="upper left")
ax_microlabel.legend(handler_map=handler_map_alpha(), loc="upper center")
# # ax.set_title("Mass fraction relative to top sub-cloud layer")

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=1, figsize=(15, 4.5))

idx = 0
for key, linestyle, color in zip(
    ["condensation", "collision_condensation", "no_physics"], ["-", "-", "-"], ["k", "k", "k"]
):
    ax = axs[idx]
    combined_output = subdata_dirs[key]["combined_output"]

    ax.plot(
        cumsum_loss.sum("radius_bins").mean("time").T,
        combined_output["pseudo_coord3"],
        label=key,
        linestyle=linestyle,
        color=color,
        alpha=0.1,
    )
    ax.plot(
        cumsum_loss.sum("radius_bins").mean("time").quantile(0.5, "cloud_id"),
        combined_output["pseudo_coord3"],
        label=key,
        linestyle=linestyle,
        alpha=1,
        color=color,
    )
    ax.set_title(key)

    idx += 1

for ax in axs:
    # ax.set_xlim(, 0.1e-4)
    ax.set_xlabel("Number of Superdroplets")
    ax.set_ylabel("gridbox")

In [ ]:
# (combined_output['gridbox'] == combined_output["max_gridbox"]).astype(float) * (1e3 * combined_output["mass_represented"] / combined_output['pseudo_gribox_volume_full']).sortby(100-combined_output["gridbox"]).cumsum(dim = "gridbox", skipna = True)
l = (combined_output["sub_cloud_layer_mask"]).astype(float) * (
    (combined_output["max_gridbox"] == combined_output["gridbox"]).astype(float)
    * 1e3
    * combined_output["mass_represented"]
    / combined_output["pseudo_gribox_volume_full"]
).max("gridbox")
l = l.where(l != 0)
ll = l + 10 * cumsum_loss
plt.imshow(ll.sum("radius_bins").mean("time"))

In [ ]:
# plt.plot(
#     ll,
#     ll.gridbox,
# );
# plt.xlim(0, 0.2)
# plt.figure()
plt.plot(
    (ll - ll.sel(gridbox=combined_output["max_gridbox"] - 2)).sum("radius_bins").mean("time"),
    ll.gridbox,
);

''

In [ ]:
(combined_output["sub_cloud_layer_mask"]).astype(float)

Text(0.5, 1.0, 'no_physics')

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=1, figsize=(15, 4.5))

idx = 0
for key, linestyle, color in zip(
    ["condensation", "collision_condensation", "no_physics"], ["-", "-", "-"], ["k", "k", "k"]
):
    ax = axs[idx]
    combined_output = subdata_dirs[key]["combined_output"]
    combined_output = combined_output.sel(cloud_id=combined_output["max_gridbox"] < 40)
    combined_output = combined_output.sel(gridbox=slice(None, 40))
    combined_output = combined_output.sel(time=slice(400, None))
    data = combined_output["number_superdroplets"]
    data = data.where(combined_output["sub_cloud_layer_no_bottom"])
    data = data.where(data <= 1e6)
    data = data.sum("radius_bins").mean("time")
    data = data.where(data != 0)
    ax.plot(
        data.T,
        data.gridbox,
        label=key,
        linestyle=linestyle,
        color=color,
        alpha=0.1,
    )
    ax.plot(
        data.quantile(0.5, "cloud_id"),
        data.gridbox,
        label=key,
        linestyle=linestyle,
        alpha=1,
        color=color,
    )
    ax.set_title(key)

    idx += 1

for ax in axs:
    ax.set_xlim(0, 150)
    ax.set_xlabel("Number of Superdroplets")
    ax.set_ylabel("gridbox")

fig.savefig("number_superdroplets.svg")

/work/um1487/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/work/um1487/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/work/um1487/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


In [ ]:
data.plot()

In [ ]:
combined_output["pseudo_gribox_volume_full"]